In [2]:
from datetime import datetime
import re
import pandas as pd
from config.ai import ModelProvider
from core.analysis import analyze_stock, analyze_stock_no_parse
from core.news import stock_rank, update_ranked_stock_price
from crawlers.general import search_engine
from crawlers.tonghuashun.main import fetch_investment_calendar, realtime_news
from crawlers.tonghuashun.tonghuashun import get_SSE_datas, get_favored_sectors, get_glamour_stocks
from maintenance.init_action import init_stock_market_info
from maintenance.manual import batch_analyze_ranked_stock
from services.ai_service import ai_search, cite_update
from services.news_service import get_stock_news, stock_analyze_prompt
from services.tushare import find_stock_name, get_last_trade_date, get_trade_date_range, realtime_quote, stock_daily_basic, stock_performance, stock_price, trade_calendar
from utils.common import get_date_range, get_today, parse_stock_suggesion, time_now
from utils.db_utils import drop_collection, find_collection_data, store_df_to_mongodb, update_by_id
import logging
from pprint import pprint

logger = logging.getLogger(__name__)

In [ ]:

def copy_analyze():
    list_old = find_collection_data('stock_rank_0210_s_01',{},{'_id':0, 'rating': 0, 'stock': 0})
    list_new = find_collection_data('stock_rank_0210_s_02')
    print(list_new[0])
    df_old = pd.DataFrame(list_old)
    df_new = pd.DataFrame(list_new)
    df_new = pd.merge(df_new, df_old,  on="ts_code", how='inner')
    update_by_id(df_new, "stock_rank_0210_s_02")

def transfer():
    drop_collection('stock_rank_0210_s_01')
    data = find_collection_data('stock_rank_0210_s_02')
    store_df_to_mongodb(pd.DataFrame(data), 'stock_rank_0210_s_01')
    drop_collection('stock_rank_0210_s_02')


2025-02-15 10:44:46 INFO [740033333.py]: ai_search_workflow loaded


In [ ]:
# 用交易日前一天的新闻验证当天的股价
# 新闻表格当天应该收录的是前一天15:00到当天15:00的新闻
if 0:
    update_ranked_stock_price('20250211', '20250212')


In [ ]:
# 获取同花顺热门股票数据
if 0:
    data_strings = get_glamour_stocks()
    total_rise = 0

    for i in range (1, 10):
            print("{:d}: {:s}\t {:4.1f}%".format(i, data_strings[i-1][0], data_strings[i-1][1]))
            total_rise = total_rise +data_strings[i-1][1]
    average = total_rise/9
    if(average >= 0):
            print("+{:4.1f} %".format(average))
    else:
            print("{:4.1f}%".format(average))

In [ ]:
if 0:
    data_strings = get_favored_sectors()

    for i in range(1, len(data_strings)):
            print("{:d}: {:s}".format(i, data_strings[i-1]))